# Snow Pit Data Ingest
---
## Raw Data Ingest

In [22]:
import xmltodict
import pandas as pd
pd.options.display.max_columns = None

# create dict from xml
# file_path = 'pit_obs.xml'
file_path = 'all-MT.xml'
with open(file_path, "rb") as file:
    pit_data = xmltodict.parse(file)

# create dataframe from dict
pit_obs = pd.DataFrame.from_dict(pit_data['Pit_Data']['Pit_Observation'])
pit_obs.columns = [col.replace('@', '') for col in pit_obs.columns]
print(pit_obs.shape)
pit_obs.sample(3)

(500, 52)


,activities,iLayerNumber,iDepth,bld,nid,stability,aviPit,crownObs,aviLoc,incline,winDir,skiAreaPit,bcPit,testPit,windspeed,aspect,sky,windLoading,heightOfSnowpack,precip,serial,version,pitNotes,timestamp,measureFrom,depthUnits,coordType,elvUnits,longType,latType,longitude,lat,zone,north,east,rhoUnits,fractureCat,hardnessScaling,state,range,share,User,Location,Layer,Shear_Test_Result,Density_Profile,Temperature_Profile,bootPen,skiPen,surfGrainType,surfGrainSize,tempUnits
449,"Collapsing, localized; Cracking;",8,10,7.77,34829,,false,false,,27,W,false,,false,,92,,,63,,drupal-nid-34829-1636583556000,7.77-0.1,"Westerly winds were blowing today, moving lots...",1636567200000,bottom,cm,lat_long,ft,E,N,-109.953017,45.061659,12T,4990333,582430,kg_cubic_m,shear_quality,linear,MT,Beartooths,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","{'@code': 'ECT', '@sdepth': '19.0', '@depthUni...","{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,NaN,NaN,NaN,NaN,C
58,,6,90,7.77,43578,,false,false,,30,SW,false,,false,Light Breeze,45,X,yes,150,S-1,drupal-nid-43578-1651453838000,7.77-0.1,,1651422600000,bottom,cm,lat_long,m,W,N,-111.01052,45.967316,12T,5090416,499185,kg_cubic_m,shear_quality,linear,MT,Bridger Range,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ST', '@sdepth': '120.0', '@depthUn...","{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,NaN,NaN,NaN,NaN,C
255,,,,7.77,39577,,false,false,,35,,false,,false,,145,SCT,no,60,NO,drupal-nid-39577-1644197306000,7.77-0.1,,1642971600000,bottom,cm,lat_long,ft,W,N,99.5814510,64.8002580,,7186321,527624,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '', '...",NaN,NaN,"{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,NaN,NaN,NaN,NaN,F


---
## Flatten Columns

User, Location and Density_Profile can be flattened into columns and concat'd to df

In [23]:
cols_to_flatten = ['User', 'Location', 'Density_Profile']
pit_obs_2 = pit_obs.copy()

for ctf in cols_to_flatten:
    df = pd.DataFrame(pit_obs[ctf].tolist())
    df.columns = [col.replace('@', f'{ctf}_') for col in df.columns]
    pit_obs_2 = pd.concat([pit_obs_2, df], axis='columns')

pit_obs_2.drop(columns=cols_to_flatten, inplace=True)
pit_obs_2.set_index('nid', inplace=True)
pit_obs_2.sample(3)


,activities,iLayerNumber,iDepth,bld,stability,aviPit,crownObs,aviLoc,incline,winDir,skiAreaPit,bcPit,testPit,windspeed,aspect,sky,windLoading,heightOfSnowpack,precip,serial,version,pitNotes,timestamp,measureFrom,depthUnits,coordType,elvUnits,longType,latType,longitude,lat,zone,north,east,rhoUnits,fractureCat,hardnessScaling,state,range,share,Layer,Shear_Test_Result,Temperature_Profile,bootPen,skiPen,surfGrainType,surfGrainSize,tempUnits,User_measureFrom,User_depthUnits,User_tempUnits,User_coordType,User_elvUnits,User_longType,User_latType,User_longitude,User_lat,User_zone,User_north,User_east,User_rhoUnits,User_fractureCat,User_hardnessScaling,User_state,User_range,User_share,User_username,User_useSymbols,User_first,User_last,User_name,User_phone,User_prof,User_email,User_affil,Location_ID,Location_type,Location_zone,Location_ns,Location_ew,Location_east,Location_north,Location_lat,Location_longitude,Location_elv,Location_state,Location_range,Location_name,Density_Profile_profile,Density_Profile_densityUnits,Density_Profile_depthUnits
nid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36016,,6,31,7.77,,false,false,,29,S,false,,false,Light Breeze,32,CLR,previous,145,NO,drupal-nid-36016-1639795889000,7.77-0.1,"After a dry and mild fall, Montana and Idaho e...",1639771200000,bottom,cm,lat_long,ft,W,N,-113.973427,45.703597,12T,5065417,268537,kg_cubic_m,fracture_character,linear,MT,Bitterroot,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'CT', '@sdepth': '125.0', '@depthUn...",NaN,100,30,NaN,NaN,C,bottom,cm,C,lat_long,ft,W,N,-113.973427,45.703597,12T,5065417,268537,kg_cubic_m,fracture_character,linear,MT,Bitterroot,true,geoffrey.fast@gmail.com,true,Geoff,Fast,Geoff Fast,406-531-7070,true,geoffrey.fast@gmail.com,West Central Montana Avalanche Center,,LATLONG,12T,N,W,268537,5065417,45.703597,-113.973427,8080,MT,Bitterroot,20211217 Lost Trail,,kg_cubic_m,cm
39377,We skied slope;,,,7.77,,false,false,,35,S,false,,false,Calm,45,OVC,yes,275,S-1,drupal-nid-39377-1643947689000,7.77-0.1,,1643916120000,bottom,cm,lat_long,ft,W,N,-115.711441,47.522958,11T,5264086,597003,kg_cubic_m,shear_quality,linear,MT,Bitterroot,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'CT', '@sdepth': '260.0', '@depthUn...",NaN,55,20,MFcl,NaN,C,bottom,cm,C,lat_long,ft,W,N,-115.711441,47.522958,11T,5264086,597003,kg_cubic_m,shear_quality,linear,MT,Bitterroot,true,michaelchurch,true,Mikey,Church,Mikey Church,406-540-3630,true,mikeylikesbikes@yahoo.com,Idaho Panhandle Avalanche Center,,LATLONG,11T,N,W,597003,5264086,47.522958,-115.711441,6307,MT,Bitterroot,Burk Summit,,kg_cubic_m,cm
39580,,4,28,7.77,,false,false,,35,,false,,false,,145,SCT,,60,NO,drupal-nid-39580-1644199507000,7.77-0.1,,1642971600000,bottom,cm,lat_long,ft,W,N,,,,,,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'CT', '@sdepth': '28.0', '@depthUni...",NaN,NaN,NaN,NaN,NaN,F,bottom,cm,F,lat_long,ft,W,N,,,,,,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,L_Buotte,true,linden,buotte,linden buotte,14065894379,false,lindenbuotte@bssd72.org,,,LATLONG,,N,W,,,,,7280,MT,Madison Range-S,solar slope,,kg_cubic_m,cm


In [24]:
pit_obs_2.User_username.value_counts()

amarienthal      58
ianhoyer         41
davidzinn        34
hahamarshall     30
nata.deleeuw     29
                 ..
Nelsonvt          1
sallyrose         1
PaulHeffernan     1
Seberia8          1
aschoening        1
Name: User_username, Length: 93, dtype: int64

---
## Split Layer and Shear_Test_Result into separate dataframes

In [10]:
layers = pd.DataFrame()

for pit_index, layer_list in pit_obs_2.Layer.items():
    if isinstance(layer_list, dict): # in case pit has only 1 layer, which will be a dict instead of a list
        layer_list = [layer_list]
    df = pd.DataFrame(layer_list)
    df['nid'] = pit_index
    layers = pd.concat([layers, df], axis='rows')

layers.columns = [col.replace('@', '') for col in layers.columns]
layers.head(10)

,grainSizeUnits1,grainSizeUnits2,grainSize,grainSize1,grainType,grainType1,hardness1,hardness2,layerNumber,waterContent,startDepth,endDepth,multipleHardness,multipleGrainType,multipleGrainSize,nid
0,mm,mm,1,,DF,,4F,,1,D,88,65,false,false,false,45124
1,mm,mm,,,MFcr,,1F,,2,D,65,64,false,false,false,45124
2,mm,mm,,,DF,FC,1F-,,3,D,64,52,false,true,false,45124
3,mm,mm,1,2,FC,,4F-,,4,D,52,45,false,false,false,45124
4,mm,mm,1,2,FC,,1F,,5,D,45,12,false,false,false,45124
5,mm,mm,,,MFcr,,P,,6,D,12,5,false,false,false,45124
6,mm,mm,,,FC,,1F-,,7,D-M,5,2,false,false,false,45124
7,mm,mm,,,MFcr,,P,,8,M,2,0,false,false,false,45124
0,mm,mm,,,MF,,F,,1,M-W,340,335,false,false,false,43627
1,mm,mm,,,MFcr,,1F,,2,,335,334,false,false,false,43627


In [13]:
shear_test_results = pd.DataFrame()

for pit_index, shear_test_result_list in pit_obs_2.Shear_Test_Result.items():
    if isinstance(shear_test_result_list, dict): # in case pit has only 1 shear test, which will be a dict instead of a list
        shear_test_result_list = [shear_test_result_list]
    if isinstance(shear_test_result_list, list):
        df = pd.DataFrame(shear_test_result_list)
        df['nid'] = pit_index
        shear_test_results = pd.concat([shear_test_results, df], axis='rows')

shear_test_results.columns = [col.replace('@', '') for col in shear_test_results.columns]
shear_test_results.sample(3)

# are character and fractureCharacter the same?

,code,sdepth,depthUnits,score,ecScore,ctScore,quality,dateString,numberOfTaps,fractureCat,fractureCharacter,s,lengthOfCut,lengthOfColumn,releaseType,dataCode,nid,character
4,ECT,78.0,cm,ECTN,10,,,12/30/2019,10,fracture_character,,ECTN10 78.0 12/30/2019.13:15:00:0,,,,,19540,NaN
4,ECT,97.0,cm,ECTN,26,,,01/23/2017,26,shear_quality,,ECTN26 97.0 01/23/2017.13:30:00:0,,,,,1429,NaN
0,ECT,128.0,cm,ECTN,14,,,02/04/2020,14,fracture_character,,ECTN14 128.0 02/04/2020.13:30:00:0,,,,,21813,NaN
